In [ ]:
# allocation Coding 2
배정 Coding
○환산피순으로 근거리 고객 한명씩 배정
 1.엑셀자료 불러오기
 2.FC별 money순으로 근거리 고객 배정
  - 근거리 : 자표거리 계산
  a.money 기준 배정량 조정
  b.지역/권역별 배정 (추후, 지역/권역별 부족시 Test)

In [1]:
# 0.환경설정
# 0.1라이브러리 불러오기
import numpy as np
import pandas as pd
import os
import numbers
import math

In [2]:
# 1.엑셀자료 불러오기
base_dir = 'C:/Users/xnote/Documents/py'
excel_file = 'allocation Coding 2.xlsx'
excel_dir = os.path.join(base_dir, excel_file)

In [36]:
# FC
em01 = pd.read_excel(excel_dir, # write your directory here
                              sheet_name = 'em', 
                              header = 0, 
                              #names = ['region', 'sales_representative', 'sales_amount'], 
#                               dtype = {'region': str, 
#                                          'sales_representative': np.int64, 
#                                          'sales_amount': float}, # dictionary type
#                               index_col = 'em_num',
                              na_values = 'NaN'
#                               thousands = ',', 
#                               nrows = 10, 
                              )
em01

,em_num,em_name,money,region,latitude,hardness
0,1,A,14,수도권,126.861717,37.874623
1,2,B,9,수도권,126.569316,37.122076
2,3,D,55,수도권,126.806583,37.531962
3,4,E,75,수도권,126.058275,37.535272
4,5,F,9,수도권,126.775502,37.807801
5,6,G,83,수도권,126.666107,37.427021
6,7,H,73,수도권,126.244399,37.906478
7,8,J,96,수도권,126.027184,37.185274
8,9,I,44,수도권,126.224037,37.465374
9,10,K,24,충청도,126.639000,37.812000


In [4]:
# 고객
cu01 = pd.read_excel(excel_dir, # write your directory here
                              sheet_name = 'cu', 
                              header = 0, 
                              #names = ['region', 'sales_representative', 'sales_amount'], 
#                               dtype = {'region': str, 
#                                          'sales_representative': np.int64, 
#                                          'sales_amount': float}, # dictionary type
#                               index_col = 'cu_num',
                              na_values = 'NaN'
#                               thousands = ',', 
#                               nrows = 10, 
                              )
print(cu01)

    cu_num cu_name region    latitude   hardness
0        1       A    수도권  126.744060  37.380985
1        2       B    수도권  126.228165  37.208757
2        3       D    수도권  126.797383  37.293906
3        4       E    수도권  126.567757  37.937422
4        5       F    수도권  126.840459  37.341861
5        6       G    수도권  126.701225  37.062907
6        7       H    수도권  126.183110  37.059070
7        8       J    수도권  126.423235  37.208687
8        9       I    수도권  126.964673  37.674086
9       10       K    수도권  126.652575  37.954124
10      11       C    수도권  126.885017  37.204106
11      12       L    수도권  126.788368  37.265269
12      13       M    수도권  126.582492  37.545942
13      14       N    수도권  126.411957  37.718648
14      15       O    수도권  126.410547  37.233936
15      16       P    수도권  126.966812  37.040045
16      17       Q    수도권  126.449354  37.050561
17      18       R    수도권  126.243210  37.200198
18      19       S    수도권  126.401271  37.551730
19      20       T  

In [5]:
#  2.FC별 환산피순으로 근거리 고객 배정
#   - 근거리 : 자표거리 계산

# 2.1  거리함수 선정
class GeoUtil:
    """
    Geographical Utils
    """
    @staticmethod
    def degree2radius(degree):
        return degree * (math.pi/180)
    
    @staticmethod
    def get_harversion_distance(x1, y1, x2, y2, round_decimal_digits=5):
        """
        경위도 (x1,y1)과 (x2,y2) 점의 거리를 반환
        Harversion Formula 이용하여 2개의 경위도간 거래를 구함(단위:Km)
        """
        if x1 is None or y1 is None or x2 is None or y2 is None:
            return None
        assert isinstance(x1, numbers.Number) and -180 <= x1 and x1 <= 180
        assert isinstance(y1, numbers.Number) and  -90 <= y1 and y1 <=  90
        assert isinstance(x2, numbers.Number) and -180 <= x2 and x2 <= 180
        assert isinstance(y2, numbers.Number) and  -90 <= y2 and y2 <=  90

        R = 6371 # 지구의 반경(단위: km)
        dLon = GeoUtil.degree2radius(x2-x1)    
        dLat = GeoUtil.degree2radius(y2-y1)

        a = math.sin(dLat/2) * math.sin(dLat/2) \
            + (math.cos(GeoUtil.degree2radius(y1)) \
              *math.cos(GeoUtil.degree2radius(y2)) \
              *math.sin(dLon/2) * math.sin(dLon/2))
        b = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
        return round(R * b, round_decimal_digits)

    @staticmethod
    def get_euclidean_distance(x1, y1, x2, y2, round_decimal_digits=5):        
        """
        유클리안 Formula 이용하여 (x1,y1)과 (x2,y2) 점의 거리를 반환
        """
        if x1 is None or y1 is None or x2 is None or y2 is None:
            return None
        assert isinstance(x1, numbers.Number) and -180 <= x1 and x1 <= 180
        assert isinstance(y1, numbers.Number) and  -90 <= y1 and y1 <=  90
        assert isinstance(x2, numbers.Number) and -180 <= x2 and x2 <= 180
        assert isinstance(y2, numbers.Number) and  -90 <= y2 and y2 <=  90

        dLon = abs(x2-x1) # 경도 차이
        if dLon >= 180:   # 반대편으로 갈 수 있는 경우
            dLon -= 360   # 반대편 각을 구한다
        dLat = y2-y1      # 위도 차이
        return round(math.sqrt(pow(dLon,2)+pow(dLat,2)),round_decimal_digits)

In [52]:
#  2.FC별 환산피순으로 근거리 고객 배정
#   - 근거리 : 자표거리 계산

# 2.2 FC별 money순 정렬
# 배정 고객수 
# money 기준마다 배정수 생성 30원 미만 1개, 60원 미만 2개, 그외 3개
em02 = em01[em01['money'] >= 30]; em02['all_cnt'] = 2;
em03 = em01[em01['money'] >= 60]; em03['all_cnt'] = 3;
em01['all_cnt'] = 1;
em04 = pd.concat([em01, em02, em03]
                 , ignore_index = True)
em04.sort_values(by=['money', 'em_num'], axis=0, ascending = False, inplace = True)
print(em04)

# 2.3 FC별 최소거리 고객 배정
## 데이터 프레임 길이구해서 for range 값에 추가 -> 결과를 FC 데이터에 결합해서 배정 데이터 생성
i = 0; j = 0;
len1 = (len(em01)); len2 = (len(cu01));
l1 = []; dis1 = []; mindis1 = [];
all01 = pd.DataFrame({'em_num':[], 'cu_num':[], 'distance':[] })

for i in range(0, len1):
    em_list1 = list(em01.loc[i])
    dis1 = []
#     print("i", i)
#     print(em_list1)
    for j in range(0, len2):
#         print("j", j)
        dis1.append(GeoUtil.get_harversion_distance(cu01.loc[i,"latitude"], cu01.loc[i,"hardness"]
                                                    , cu01.loc[j,"latitude"], cu01.loc[j,"hardness"])) #  거리계산
    

    em_num em_name  money region    latitude   hardness  all_cnt
0        8       J     96    수도권  126.027184  37.185274        1
20       8       J     96    수도권  126.027184  37.185274        2
32       8       J     96    수도권  126.027184  37.185274        3
1       15       F     85    충청도  126.580000  37.286000        1
21      15       F     85    충청도  126.580000  37.286000        2
33      15       F     85    충청도  126.580000  37.286000        3
2        6       G     83    수도권  126.666107  37.427021        1
22       6       G     83    수도권  126.666107  37.427021        2
34       6       G     83    수도권  126.666107  37.427021        3
3        4       E     75    수도권  126.058275  37.535272        1
23       4       E     75    수도권  126.058275  37.535272        2
35       4       E     75    수도권  126.058275  37.535272        3
4        7       H     73    수도권  126.244399  37.906478        1
24       7       H     73    수도권  126.244399  37.906478        2
36       7       H     73

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [48]:
em02 = em01[em01['money'] >= 30]; em02['all_cnt'] = 2;
em03 = em01[em01['money'] >= 60]; em03['all_cnt'] = 3;


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,em_num,em_name,money,region,latitude,hardness,all_cnt
3,4,E,75,수도권,126.058275,37.535272,3
5,6,G,83,수도권,126.666107,37.427021,3
6,7,H,73,수도권,126.244399,37.906478,3
7,8,J,96,수도권,126.027184,37.185274,3
14,15,F,85,충청도,126.580000,37.286000,3
15,16,G,62,충청도,125.893000,37.687000,3
18,19,I,64,충청도,126.159000,37.257000,3


In [49]:
em02 = em01[em01['money'] >= 30]; em02.loc['all_cnt'] = 2;
em02

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,em_num,em_name,money,region,latitude,hardness
2,3,D,55,수도권,126.806583,37.531962
3,4,E,75,수도권,126.058275,37.535272
5,6,G,83,수도권,126.666107,37.427021
6,7,H,73,수도권,126.244399,37.906478
7,8,J,96,수도권,126.027184,37.185274
8,9,I,44,수도권,126.224037,37.465374
11,12,B,47,충청도,125.823000,37.250000
12,13,D,33,충청도,126.549000,37.003000
14,15,F,85,충청도,126.580000,37.286000
15,16,G,62,충청도,125.893000,37.687000


In [ ]:
#  2.FC별 환산피순으로 근거리 고객 배정
#   - 근거리 : 자표거리 계산

# 2.2 FC별 환산피순 정렬
em01.sort_values(by=['money'], axis=0, ascending = False, inplace = True)

# 2.3 FC별 최소거리 고객 배정
## 데이터 프레임 길이구해서 for range 값에 추가 -> 결과를 FC 데이터에 결합해서 배정 데이터 생성
i = 0; j = 0;
len1 = (len(em01)); len2 = (len(cu01));
l1 = []; dis1 = []; mindis1 = [];
all01 = pd.DataFrame({'em_num':[], 'cu_num':[], 'distance':[] })

for i in range(0, len1):
    em_list1 = list(em01.loc[i])
    dis1 = []
#     print("i", i)
#     print(em_list1)
    for j in range(0, len2):
#         print("j", j)
        dis1.append(GeoUtil.get_harversion_distance(em_list1[3], em_list1[4], cu01.loc[j,"latitude"], cu01.loc[j,"hardness"])) #  거리계산
    
#     all01 = pd.concat([all01, pd.DataFrame({'em_num':em_list1[0], 'cu_num':cu01.loc[dis1.index(max(dis1)-1),"cu_num"]])
    print(dis1)
    print(min(dis1))

    all01 = pd.concat([ all01, pd.DataFrame({'em_num':em_list1[0], 'cu_num':[cu01.loc[dis1.index(min(dis1)),"cu_num"]] 
                     , 'distance':min(dis1) }) ]
                     , ignore_index = True)
print(all01)

In [98]:
# 결과확인 : FC에 배정데이터 결합
final01 = pd.merge(em01, all01, how = 'inner', on = 'em_num')
final01

,em_num,em_name,money,latitude,hardness,cu_num,distance
0,10,K,99,126.867068,37.205859,11.0,1.60158
1,2,B,48,126.569316,37.122076,17.0,13.28416
2,1,A,41,126.861717,37.874623,10.0,20.36569
3,4,E,30,126.058275,37.535272,19.0,30.29565
4,5,F,21,126.775502,37.807801,10.0,19.52236
5,6,G,19,126.666107,37.427021,1.0,8.57989
6,7,H,17,126.244399,37.906478,14.0,25.55142
7,8,J,15,126.027184,37.185274,2.0,17.99214
8,9,I,15,126.224037,37.465374,19.0,18.34672
9,3,D,5,126.806583,37.531962,1.0,17.67178


In [73]:
cu01.loc[dis1.index(max(dis1))]

cu_num           20
cu_name           T
latitude     126.58
hardness    37.9929
Name: 19, dtype: object

In [75]:
cu01.loc[dis1.index(max(dis1))-1,"cu_num"]

19

In [60]:
cu01.loc[dis1.index(max(dis1))+1,"cu_num"]

4

In [13]:
dis1.index(max(dis1))

1

In [38]:
t1 = pd.DataFrame([1,2])
t2 = pd.DataFrame([1,2])
pd.concat([t1, t2])


,0
0,1
1,2
0,1
1,2


In [51]:
# df01 = pd.DataFrame({'em_num':[1], 'cu_num':[1]}, index=['2'])
# df01 = pd.DataFrame({'em_num':[1], 'cu_num':[1]})
df01 = pd.DataFrame({'em_num':[], 'cu_num':[]})
df01

,em_num,cu_num


In [8]:
# 서울시청 126.97843, 37.56668
# 강남역   127.02758, 37.49794
# 방법1)
GeoUtil.get_euclidean_distance(126.97843, 37.56668, 127.02758, 37.49794)





0.0845

In [9]:
# 방법2)
GeoUtil.get_harversion_distance(126.97843, 37.56668, 127.02758, 37.49794)


8.78676

In [ ]:


class GeoUtil:
    """
    Geographical Utils
    """
    @staticmethod
    def degree2radius(degree):
        return degree * (math.pi/180)
    
    @staticmethod
    def get_harversion_distance(x1, y1, x2, y2, round_decimal_digits=5):
        """
        경위도 (x1,y1)과 (x2,y2) 점의 거리를 반환
        Harversion Formula 이용하여 2개의 경위도간 거래를 구함(단위:Km)
        """
        if x1 is None or y1 is None or x2 is None or y2 is None:
            return None
        assert isinstance(x1, numbers.Number) and -180 <= x1 and x1 <= 180
        assert isinstance(y1, numbers.Number) and  -90 <= y1 and y1 <=  90
        assert isinstance(x2, numbers.Number) and -180 <= x2 and x2 <= 180
        assert isinstance(y2, numbers.Number) and  -90 <= y2 and y2 <=  90

        R = 6371 # 지구의 반경(단위: km)
        dLon = GeoUtil.degree2radius(x2-x1)    
        dLat = GeoUtil.degree2radius(y2-y1)

        a = math.sin(dLat/2) * math.sin(dLat/2) \
            + (math.cos(GeoUtil.degree2radius(y1)) \
              *math.cos(GeoUtil.degree2radius(y2)) \
              *math.sin(dLon/2) * math.sin(dLon/2))
        b = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
        return round(R * b, round_decimal_digits)

    @staticmethod
    def get_euclidean_distance(x1, y1, x2, y2, round_decimal_digits=5):        
        """
        유클리안 Formula 이용하여 (x1,y1)과 (x2,y2) 점의 거리를 반환
        """
        if x1 is None or y1 is None or x2 is None or y2 is None:
            return None
        assert isinstance(x1, numbers.Number) and -180 <= x1 and x1 <= 180
        assert isinstance(y1, numbers.Number) and  -90 <= y1 and y1 <=  90
        assert isinstance(x2, numbers.Number) and -180 <= x2 and x2 <= 180
        assert isinstance(y2, numbers.Number) and  -90 <= y2 and y2 <=  90

        dLon = abs(x2-x1) # 경도 차이
        if dLon >= 180:   # 반대편으로 갈 수 있는 경우
            dLon -= 360   # 반대편 각을 구한다
        dLat = y2-y1      # 위도 차이
        return round(math.sqrt(pow(dLon,2)+pow(dLat,2)),round_decimal_digits)

class GeoUtil:
    """
    Geographical Utils
    """
    @staticmethod
    def degree2radius(degree):
        return degree * (math.pi/180)
    
    @staticmethod
    def get_harversion_distance(x1, y1, x2, y2, round_decimal_digits=5):
        """
        경위도 (x1,y1)과 (x2,y2) 점의 거리를 반환
        Harversion Formula 이용하여 2개의 경위도간 거래를 구함(단위:Km)
        """
        if x1 is None or y1 is None or x2 is None or y2 is None:
            return None
        assert isinstance(x1, numbers.Number) and -180 <= x1 and x1 <= 180
        assert isinstance(y1, numbers.Number) and  -90 <= y1 and y1 <=  90
        assert isinstance(x2, numbers.Number) and -180 <= x2 and x2 <= 180
        assert isinstance(y2, numbers.Number) and  -90 <= y2 and y2 <=  90

        R = 6371 # 지구의 반경(단위: km)
        dLon = GeoUtil.degree2radius(x2-x1)    
        dLat = GeoUtil.degree2radius(y2-y1)

        a = math.sin(dLat/2) * math.sin(dLat/2) \
            + (math.cos(GeoUtil.degree2radius(y1)) \
              *math.cos(GeoUtil.degree2radius(y2)) \
              *math.sin(dLon/2) * math.sin(dLon/2))
        b = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
        return round(R * b, round_decimal_digits)

    @staticmethod
    def get_euclidean_distance(x1, y1, x2, y2, round_decimal_digits=5):        
        """
        유클리안 Formula 이용하여 (x1,y1)과 (x2,y2) 점의 거리를 반환
        """
        if x1 is None or y1 is None or x2 is None or y2 is None:
            return None
        assert isinstance(x1, numbers.Number) and -180 <= x1 and x1 <= 180
        assert isinstance(y1, numbers.Number) and  -90 <= y1 and y1 <=  90
        assert isinstance(x2, numbers.Number) and -180 <= x2 and x2 <= 180
        assert isinstance(y2, numbers.Number) and  -90 <= y2 and y2 <=  90

        dLon = abs(x2-x1) # 경도 차이
        if dLon >= 180:   # 반대편으로 갈 수 있는 경우
            dLon -= 360   # 반대편 각을 구한다
        dLat = y2-y1      # 위도 차이
        return round(math.sqrt(pow(dLon,2)+pow(dLat,2)),round_decimal_digits)